In [56]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
import json

class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def printDetails(self):
        print("\tINPUT SIZE:",self.input_size)
        print("\tLAYERS:", self.layers)
        print("\tINPUT:", self.input)
        print("\tINITIAL WEIGHTS:", self.initial_weights)
        print("\tTARGET:", self.target)
        print("\tLEARNING RATE:", self.learning_rate)
        print("\tBATCH SIZE:", self.batch_size)
        print("\tMAX ITERATION:", self.max_iteration)
        print("\tERROR THRESHOLD:", self.error_threshold)
    

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_size = self.model.get('input_size')

            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            
            self.input = self.case.get('input', [])
            print(self.case.get("initial_weights"))
            self.initial_weights = self.case.get('initial_weights', [])
            self.target = self.case.get('target', [])
            self.parameters = self.case.get('learning_parameters', {})

            self.learning_rate = self.parameters.get('learning_rate')
            self.batch_size = self.parameters.get('batch_size')
            self.max_iteration = self.parameters.get('max_iteration')
            self.error_threshold = self.parameters.get('error_threshold')

            self.expect = self.data.get('expect', {})
            self.stopped_by = self.expect.get('stopped_by', '')
            self.final_weights = self.expect.get('final_weights', [])

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")

In [56]:
import tensorflow as tf

class TensorFlowModel: 
    def __init__(self, inputs,targets, number_of_neurons, learning_rate, batch_size, activation, initial_weights) -> None:
        self.inputs = tf.constant(inputs,dtype=tf.float32)
        self.targets = tf.constant(targets, dtype=tf.float32)
        self.initial_weights = tf.constant(initial_weights, dtype=tf.float32)
        self.input_size = len(inputs[0])
        self.number_of_neurons = number_of_neurons 
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.dataset = tf.data.Dataset.from_tensor_slices((self.inputs, self.targets))
        self.dataset = self.dataset.batch(batch_size)  # Batch the data
        self.initial_bias = tf.ones([number_of_neurons], dtype=tf.float32)
        self.activation = activation
        self.model = tf.keras.Sequential([
                        tf.keras.layers.Dense(number_of_neurons,
                                            input_shape=(self.input_size,),
                                            activation=self.activation,
                                            use_bias=True)
                    ])
        self.model.layers[0].set_weights([self.initial_weights, self.initial_bias])
    
    def train_step(self, inputs, target):
        optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)
        if self.activation == "softmax":
            loss_function = tf.keras.losses.CategoricalCrossentropy()
        else:
            loss_function = tf.keras.losses.MeanSquaredError()

        with tf.GradientTape() as tape:
            predictions = self.model(inputs, training=True)
            loss = loss_function(target, predictions)

        # Only update the weights (first variable in trainable_variables)
        weights = self.model.trainable_variables[0]  # This assumes the first variable is the weight matrix
        gradients = tape.gradient(loss, [weights])
        optimizer.apply_gradients(zip(gradients, [weights]))
        return loss

    def show_prediction(self):
        print("========================================")
        print(self.model.layers)
        updated_weights = self.model.layers[0].get_weights()[0]
        print("Updated weights:\n", updated_weights)
        print("=========================================")

    def predict(self,max_iteration):
        for i in range(max_iteration):
            # Training loop
            for inputs_batch, targets_batch in self.dataset:
                loss = self.train_step(inputs_batch, targets_batch)
                print("Total loss : ", loss, "\n")

        self.show_prediction()


In [59]:
import os

def main():
    currDir = os.getcwd()
    testDir = currDir.replace("src","test")
    file = os.path.join(testDir, "linear_two_iteration.json")
    json_file = JsonModelParser(file)
    inputs = json_file.input
    if(inputs[0] != json_file.input_size):
        # Kasus kalau misalnay ada bias 
        inputs = [[1] + sub_array for sub_array in json_file.input]
    targets = json_file.target
    number_of_neurons = json_file.layers[0]["number_of_neurons"]
    learning_rate = json_file.learning_rate
    batch_size = json_file.batch_size
    activation = json_file.layers[0]["activation_function"]
    initial_weights = json_file.initial_weights[0]
    max_iter = json_file.max_iteration
    model = TensorFlowModel(inputs, targets, number_of_neurons, learning_rate, batch_size, activation, initial_weights)
    model.predict(max_iter)

main()

[[[0.1, 0.3, 0.2], [0.4, 0.2, -0.7], [0.1, -0.8, 0.5]]]
Total loss :  tf.Tensor(0.92166674, shape=(), dtype=float32) 

Total loss :  tf.Tensor(0.24347591, shape=(), dtype=float32) 

[<Dense name=dense_25, built=True>]
Updated weights:
 [[ 0.05955555  0.22977777  0.05366666]
 [ 0.32133335  0.05066667 -0.991     ]
 [ 0.03822222 -0.9008889   0.27966666]]


In [37]:

inputs = [[1.0, 3.0, 1.0], [1.0, 1.0, 2.0]]
targets = [[2.0, 0.3, -1.9], [1.3, -0.7, 0.1]]
number_of_neurons = 3
learning_rate = 0.1
batch_size = 2
activation = "linear" 
initial_weights = [
              [0.1, 0.3, 0.2],
              [0.4, 0.2, -0.7],
              [0.1, -0.8, 0.5]
            ]
max_iteration = 3
model = TensorFlowModel(inputs, targets, number_of_neurons, learning_rate, batch_size, activation, initial_weights)
model.predict(1)

Updated weights:
 [[ 0.07333332  0.25333333  0.10333333]
 [ 0.34666666  0.1        -0.89666665]
 [ 0.05999999 -0.8666667   0.35666665]]


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
